# Obesity Rate Correlation Analysis

* The analysis will encompass data from the states of California, Ohio, and South Carolina.
* The objective is to find if there is correlation between obesity rate and the following measurements: 
    * Median household income
    * % With access to exercise opportunities
    * % of physically inactive
    * Primary care physicians rate
    * High School Unemployment rate
    * Unemployed rate
    

In [1]:
import requests
import pandas as pd
import csv

In [2]:
# Countyhealthrankings data for California, Ohio and South Carolina
# source https://www.countyhealthrankings.org/

csvpath_california = "./2020 County Health Rankings California Data - v1_0.csv"
csvpath_ohio = "./2020 County Health Rankings Ohio Data - v1_0.csv"
csvpath_south_carolina = "./2020 County Health Rankings South Carolina Data - v1_0.csv"

In [3]:
# Census URL to collect median household income for 2018 for every state and county

url = "https://api.census.gov/data/2018/acs/acs5?get=NAME,B19013_001E&for=county:*&in=state:*"
print(url)

https://api.census.gov/data/2018/acs/acs5?get=NAME,B19013_001E&for=county:*&in=state:*


## Data Cleaning
* Cleaning Data from the US Census and countyhealthrankings for the following states:
    * California 
    * Ohio
    * South Carolina

In [4]:
# Converting census request into a DataFrame

census_data = requests.get(url).json()
df = pd.DataFrame(census_data).drop([0,])
df.head()

,0,1,2,3
1,"Washington County, Mississippi",30834,28,151
2,"Perry County, Mississippi",39007,28,111
3,"Choctaw County, Mississippi",37203,28,019
4,"Itawamba County, Mississippi",40510,28,057
5,"Carroll County, Mississippi",43060,28,015


In [5]:
# Renaming column headers from US Census data

new_df = df.rename(columns = {0:"County and State",
                             1:"Median Income",
                             2:"State Code",
                             3: "County Code"})
new_df.head()

,County and State,Median Income,State Code,County Code
1,"Washington County, Mississippi",30834,28,151
2,"Perry County, Mississippi",39007,28,111
3,"Choctaw County, Mississippi",37203,28,019
4,"Itawamba County, Mississippi",40510,28,057
5,"Carroll County, Mississippi",43060,28,015


In [6]:
# Combining State and County Code in order to merge the DataFrame with the Countyhealthrankings
# DataFrame

new_df["FIPS"] = new_df["State Code"].astype(str) + new_df["County Code"].astype(str)
new_df.head()

,County and State,Median Income,State Code,County Code,FIPS
1,"Washington County, Mississippi",30834,28,151,28151
2,"Perry County, Mississippi",39007,28,111,28111
3,"Choctaw County, Mississippi",37203,28,019,28019
4,"Itawamba County, Mississippi",40510,28,057,28057
5,"Carroll County, Mississippi",43060,28,015,28015


In [7]:
# Splitting the County and State column and creating two new columns for "County" and "State"
# to filter by California, Ohio and South Carolina

split_county_state = new_df["County and State"].str.split(",", n = 0, expand = True) 
new_df["County"]= split_county_state[0]
new_df["State"] = split_county_state[1]
new_df.head()

,County and State,Median Income,State Code,County Code,FIPS,County,State
1,"Washington County, Mississippi",30834,28,151,28151,Washington County,Mississippi
2,"Perry County, Mississippi",39007,28,111,28111,Perry County,Mississippi
3,"Choctaw County, Mississippi",37203,28,019,28019,Choctaw County,Mississippi
4,"Itawamba County, Mississippi",40510,28,057,28057,Itawamba County,Mississippi
5,"Carroll County, Mississippi",43060,28,015,28015,Carroll County,Mississippi


In [64]:
# Filtering data for the states of California, Ohio and South Carolina
# Selecting only needed columns for "County and State","Median Income","State and County Code", and "State"
selected_states = []
clean_data = new_df[["County and State","Median Income","FIPS","State"]].sort_values(by=['FIPS'])
states = [" California"," Ohio"," South Carolina"]
selected_states = clean_data.loc[(clean_data["State"]==" California") | (clean_data["State"]==" Ohio") | (clean_data["State"]==" South Carolina") ]
selected_states.head()

,County and State,Median Income,FIPS,State
1864,"Alameda County, California",92574,06001,California
1872,"Alpine County, California",64688,06003,California
1893,"Amador County, California",61198,06005,California
1884,"Butte County, California",48443,06007,California
1888,"Calaveras County, California",58151,06009,California


In [75]:
# create a copy of the selected_states DataFrame in order to cast the FIPS column as integer

selected_states_copy = selected_states.copy()
selected_states_copy["FIPS"] = selected_states["FIPS"].astype(int)
selected_states_copy.dtypes

County and State    object
Median Income       object
FIPS                 int64
State               object
dtype: object

In [78]:
# Read all csv files for California, Ohio and South Carolina.
# Extract only columns needed for analysis

california_df = pd.read_csv(csvpath_california)
ohio_df = pd.read_csv(csvpath_ohio)
south_carolina_df = pd.read_csv(csvpath_south_carolina)

ca_health_df = california_df[["FIPS","% Adults with Obesity","% Physically Inactive","% With Access to Exercise Opportunities","Primary Care Physicians Rate","High School Graduation Rate","% Unemployed"]]
ohio_health_df = ohio_df[["FIPS","% Adults with Obesity","% Physically Inactive","% With Access to Exercise Opportunities","Primary Care Physicians Rate","High School Graduation Rate","% Unemployed"]]
south_carolina_health_df = south_carolina_df[["FIPS","% Adults with Obesity","% Physically Inactive","% With Access to Exercise Opportunities","Primary Care Physicians Rate","High School Graduation Rate","% Unemployed"]]


In [52]:
# Concatenating all health data from countyhealth rankings for the three states into one DataFrame

all_states = [ca_health_df,ohio_health_df,south_carolina_health_df]
state_health_df = pd.concat(all_states)
state_health_df

,FIPS,% Adults with Obesity,% Physically Inactive,% With Access to Exercise Opportunities,Primary Care Physicians Rate,High School Graduation Rate,% Unemployed
0,6000,24,18,93,79,83.0,4.2
1,6001,19,15,100,110,87.0,3.0
2,6003,23,21,100,0,NaN,4.6
3,6005,30,25,76,67,88.0,4.0
4,6007,27,22,84,61,85.0,5.0
...,...,...,...,...,...,...,...
42,45083,32,28,77,68,88.0,3.1
43,45085,38,28,69,51,84.0,4.0
44,45087,39,30,65,40,76.0,4.1
45,45089,42,34,20,16,86.0,5.4


In [15]:
state_health_df.astype({'FIPS': 'str'}).dtypes

FIPS                                        object
% Adults with Obesity                        int64
% Physically Inactive                        int64
% With Access to Exercise Opportunities      int64
Primary Care Physicians Rate                 int64
High School Graduation Rate                float64
% Unemployed                               float64
dtype: object

In [20]:
# Add 0 to the front of State/County code
# Used apply function along with format. Apply passes a function and applies it to every value in the series. 
# {:0>5}'.format | padding and aligning strings. :0 refers to the character we want to use to pad, in this case 0. 
# > tells it to align to the right. The 5 is the length of the returned character/digit. 
# In this case since the FIPS is only 4 digits this will apply one 0 to the front, making it a total of 5 digits
# https://www.w3resource.com/python/python-format.php
# https://www.geeksforgeeks.org/python-pandas-apply/

format_FIPS = state_health_df['FIPS'].apply('{:0>5}'.format).head()

In [21]:
# Converting the formated FIPS to a DataFrame

FIPS_df = pd.DataFrame(format_FIPS)

In [22]:
# Replacing the formatted FIPS column to the DataFrame

state_health_df["FIPS"] = FIPS_df
state_health_df.head()

,FIPS,% Adults with Obesity,% Physically Inactive,% With Access to Exercise Opportunities,Primary Care Physicians Rate,High School Graduation Rate,% Unemployed
0,06000,24,18,93,79,83.0,4.2
1,06001,19,15,100,110,87.0,3.0
2,06003,23,21,100,0,NaN,4.6
3,06005,30,25,76,67,88.0,4.0
4,06007,27,22,84,61,85.0,5.0


In [79]:
# Merge data from the US census and countyhealthrankings into one DataFrame and drop NA

merged_df = pd.merge(state_health_df,selected_states_copy,on="FIPS",how="left")
merged_df.dropna()

,FIPS,% Adults with Obesity,% Physically Inactive,% With Access to Exercise Opportunities,Primary Care Physicians Rate,High School Graduation Rate,% Unemployed,County and State,Median Income,State
1,6001,19,15,100,110,87.0,3.0,"Alameda County, California",92574,California
3,6005,30,25,76,67,88.0,4.0,"Amador County, California",61198,California
4,6007,27,22,84,61,85.0,5.0,"Butte County, California",48443,California
5,6009,31,31,60,44,90.0,4.0,"Calaveras County, California",58151,California
6,6011,41,28,75,32,89.0,12.9,"Colusa County, California",56704,California
...,...,...,...,...,...,...,...,...,...,...
190,45083,32,28,77,68,88.0,3.1,"Spartanburg County, South Carolina",50179,South Carolina
191,45085,38,28,69,51,84.0,4.0,"Sumter County, South Carolina",43792,South Carolina
192,45087,39,30,65,40,76.0,4.1,"Union County, South Carolina",40120,South Carolina
193,45089,42,34,20,16,86.0,5.4,"Williamsburg County, South Carolina",32351,South Carolina


## Correlation and Regression Analysis